<a href="https://colab.research.google.com/github/amrutadeo-22/resnet/blob/main/Efficient_Net.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

# Swish activation function
def swish(x):
    return x * torch.sigmoid(x)

# Squeeze-and-Excitation (SE) Block
class SEBlock(nn.Module):
    def __init__(self, in_channels, reduction=16):
        super(SEBlock, self).__init__()
        self.fc1 = nn.Conv2d(in_channels, in_channels // reduction, kernel_size=1)
        self.fc2 = nn.Conv2d(in_channels // reduction, in_channels, kernel_size=1)

    def forward(self, x):
        scale = F.adaptive_avg_pool2d(x, 1)
        scale = F.relu(self.fc1(scale))
        scale = torch.sigmoid(self.fc2(scale))
        return x * scale

# Convolutional Block Attention Module (CBAM)
class CBAM(nn.Module):
    def __init__(self, in_channels, reduction=16):
        super(CBAM, self).__init__()
        self.se_block = SEBlock(in_channels, reduction)
        self.spatial_attention = nn.Conv2d(2, 1, kernel_size=7, stride=1, padding=3, bias=False)

    def forward(self, x):
        # Channel Attention
        x = self.se_block(x)

        # Spatial Attention
        max_pool = torch.max(x, dim=1, keepdim=True).values
        avg_pool = torch.mean(x, dim=1, keepdim=True)
        spatial_attention = torch.cat([max_pool, avg_pool], dim=1)
        spatial_attention = torch.sigmoid(self.spatial_attention(spatial_attention))

        return x * spatial_attention

# Depthwise Separable Convolution
class DepthwiseSeparableConv(nn.Module):
    def __init__(self, in_channels, out_channels, stride=1):
        super(DepthwiseSeparableConv, self).__init__()
        self.depthwise = nn.Conv2d(in_channels, in_channels, kernel_size=3, stride=stride, padding=1, groups=in_channels, bias=False)
        self.pointwise = nn.Conv2d(in_channels, out_channels, kernel_size=1, bias=False)
        self.bn = nn.BatchNorm2d(out_channels)

    def forward(self, x):
        x = self.depthwise(x)
        x = self.pointwise(x)
        x = self.bn(x)
        return F.relu(x)

# Wide ResNet Block
class WideResNetBlock(nn.Module):
    def __init__(self, in_channels, out_channels, stride=1):
        super(WideResNetBlock, self).__init__()
        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(out_channels)

        self.shortcut = nn.Sequential()
        if stride != 1 or in_channels != out_channels:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_channels, out_channels, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(out_channels)
            )

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        out += self.shortcut(x)
        return F.relu(out)

# Aggregated ResNet Block
class AggregatedResNetBlock(nn.Module):
    def __init__(self, in_channels, out_channels, cardinality=32, stride=1):
        super(AggregatedResNetBlock, self).__init__()
        self.split_channels = out_channels // cardinality
        self.convs = nn.ModuleList([
            nn.Conv2d(in_channels, self.split_channels, kernel_size=3, stride=stride, padding=1, bias=False)
            for _ in range(cardinality)
        ])
        self.bn = nn.BatchNorm2d(out_channels)

    def forward(self, x):
        split_outputs = [conv(x) for conv in self.convs]
        out = torch.cat(split_outputs, dim=1)
        out = self.bn(out)
        return F.relu(out)

# Custom Model integrating EfficientNet ideas, CBAM, Wide ResNet, and Aggregated ResNet
class CustomEfficientResNet(nn.Module):
    def __init__(self, num_classes=10):
        super(CustomEfficientResNet, self).__init__()
        self.initial_conv = nn.Conv2d(3, 32, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn = nn.BatchNorm2d(32)

        self.block1 = WideResNetBlock(32, 64, stride=1)
        self.block2 = AggregatedResNetBlock(64, 128, cardinality=32, stride=2)
        self.block3 = CBAM(128)
        self.block4 = DepthwiseSeparableConv(128, 256, stride=2)
        self.block5 = CBAM(256)

        self.pool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc = nn.Linear(256, num_classes)

    def forward(self, x):
        x = F.relu(self.bn(self.initial_conv(x)))
        x = self.block1(x)
        x = self.block2(x)
        x = self.block3(x)
        x = self.block4(x)
        x = self.block5(x)
        x = self.pool(x)
        x = torch.flatten(x, 1)
        x = self.fc(x)
        return x

# Training setup
def train(model, train_loader, criterion, optimizer, device):
    model.train()
    running_loss = 0
    correct = 0
    total = 0
    for data, targets in train_loader:
        data, targets = data.to(device), targets.to(device)
        optimizer.zero_grad()
        outputs = model(data)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
        _, predicted = outputs.max(1)
        total += targets.size(0)
        correct += predicted.eq(targets).sum().item()
    return running_loss / len(train_loader), 100. * correct / total

# Testing setup
def test(model, test_loader, criterion, device):
    model.eval()
    running_loss = 0
    correct = 0
    total = 0
    with torch.no_grad():
        for data, targets in test_loader:
            data, targets = data.to(device), targets.to(device)
            outputs = model(data)
            loss = criterion(outputs, targets)
            running_loss += loss.item()
            _, predicted = outputs.max(1)
            total += targets.size(0)
            correct += predicted.eq(targets).sum().item()
    return running_loss / len(test_loader), 100. * correct / total

# Main function
def main():
    # Hyperparameters
    batch_size = 128
    num_epochs = 10
    learning_rate = 0.001

    # Device configuration
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

    # Data augmentation and normalization
    transform = transforms.Compose([
        transforms.RandomCrop(32, padding=4),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
    ])

    # CIFAR-10 dataset
    train_dataset = datasets.CIFAR10(root='./data', train=True, transform=transform, download=True)
    test_dataset = datasets.CIFAR10(root='./data', train=False, transform=transform, download=True)

    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

    # Model, Loss, Optimizer
    model = CustomEfficientResNet(num_classes=10).to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)

    # Training and testing loop
    for epoch in range(num_epochs):
        train_loss, train_acc = train(model, train_loader, criterion, optimizer, device)
        test_loss, test_acc = test(model, test_loader, criterion, device)

        print(f"Epoch [{epoch + 1}/{num_epochs}]")
        print(f"Train Loss: {train_loss:.4f}, Train Accuracy: {train_acc:.2f}%")
        print(f"Test Loss: {test_loss:.4f}, Test Accuracy: {test_acc:.2f}%\n")

if __name__ == "__main__":
    main()


100%|██████████| 170M/170M [00:02<00:00, 77.8MB/s]


Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified
Epoch [1/10]
Train Loss: 1.5091, Train Accuracy: 44.44%
Test Loss: 1.3037, Test Accuracy: 51.85%

Epoch [2/10]
Train Loss: 1.1279, Train Accuracy: 59.48%
Test Loss: 1.1232, Test Accuracy: 58.96%

Epoch [3/10]
Train Loss: 0.9764, Train Accuracy: 65.03%
Test Loss: 1.0039, Test Accuracy: 64.37%

Epoch [4/10]
Train Loss: 0.8851, Train Accuracy: 68.52%
Test Loss: 0.9422, Test Accuracy: 65.85%

Epoch [5/10]
Train Loss: 0.8203, Train Accuracy: 70.85%
Test Loss: 1.2431, Test Accuracy: 59.57%

Epoch [6/10]
Train Loss: 0.7647, Train Accuracy: 72.87%
Test Loss: 0.9064, Test Accuracy: 68.55%

Epoch [7/10]
Train Loss: 0.7121, Train Accuracy: 74.86%
Test Loss: 0.8078, Test Accuracy: 71.52%

Epoch [8/10]
Train Loss: 0.6844, Train Accuracy: 76.17%
Test Loss: 0.8327, Test Accuracy: 71.16%

